In [ ]:
!pip install kagglehub
import kagglehub

path = kagglehub.dataset_download("msambare/fer2013")
print("Dataset path:", path)


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib
from skimage.feature import local_binary_pattern

In [3]:
# -----------------------------
# 1. Load images
# -----------------------------
def load_images_from_folder(folder):
    images, labels = [], []
    label_names = sorted(os.listdir(folder))
    label_map = {name: idx for idx, name in enumerate(label_names)}

    for label_name in label_names:
        path = os.path.join(folder, label_name)
        for file in os.listdir(path):
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (48,48))
                images.append(img)
                labels.append(label_map[label_name])
    return np.array(images), np.array(labels), label_map


In [ ]:
print("Load train...")
X_train, y_train, label_map = load_images_from_folder("/kaggle/input/fer2013/train")
print("Load test...")
X_test, y_test, _ = load_images_from_folder("/kaggle/input/fer2013/test")

print("Shapes:", X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Classes:", label_map)

# 1. LBP + kNN

In [12]:
# -----------------------------
# 2. LBP
# -----------------------------
def extract_lbp_features(images):
    features = []
    for img in images:
        lbp = local_binary_pattern(img, P=16, R=3, method="uniform")
        hist, _ = np.histogram(lbp.ravel(),
                               bins=np.arange(0, 16 + 3),
                               range=(0, 16 + 2),
                               density=True)
        features.append(hist)
    return np.array(features)

In [ ]:
print("Calculate LBP for train...")
X_train_lbp = extract_lbp_features(X_train)

print("Calculate LBP for test...")
X_test_lbp = extract_lbp_features(X_test)


In [ ]:
# -----------------------------
# 3. KNN
# -----------------------------
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(X_train_lbp, y_train)

# -----------------------------
# 4. Accuracy
# -----------------------------
y_pred = knn.predict(X_test_lbp)
acc = accuracy_score(y_test, y_pred)
print("LBP+KNN accuracy:", acc)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=label_map.keys()))


In [ ]:
# -----------------------------
# 6. Accuracy for different k
# -----------------------------
acc_scores = []
k_values = list(range(1, 15))
for k in k_values:
    knn_temp = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    knn_temp.fit(X_train_lbp, y_train)
    acc_scores.append(knn_temp.score(X_test_lbp, y_test))

plt.figure(figsize=(8,5))
plt.plot(k_values, acc_scores, marker="o")
plt.title("Accuracy vs k (LBP+KNN)")
plt.xlabel("k (neighbors)")
plt.ylabel("Accuracy")
plt.grid()
plt.show()


In [15]:
# -----------------------------
# 7. Save model
# -----------------------------
joblib.dump(knn, "lbp_knn.pkl")
print("Model saved lbp_knn.pkl")


Model saved lbp_knn.pkl


============ Тест=====================

In [ ]:
# -----------------------------
# LBP
# -----------------------------
def extract_lbp_features(images, P, R):
    features = []
    for img in images:
        lbp = local_binary_pattern(img, P=P, R=R, method="uniform")
        hist, _ = np.histogram(lbp.ravel(),
                               bins=np.arange(0, P + 3),
                               range=(0, P + 2),
                               density=True)
        features.append(hist)
    return np.array(features)

# -----------------------------
# Експеримент: різні P, R, k
# -----------------------------
P_values = [8, 16, 24]
R_values = [1, 2, 3]
k_values = [1, 3, 5, 7]

results = []

for P in P_values:
    for R in R_values:
        print(f"\n>>> Обчислюємо LBP з P={P}, R={R}")
        X_train_lbp = extract_lbp_features(X_train, P, R)
        X_test_lbp = extract_lbp_features(X_test, P, R)

        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
            knn.fit(X_train_lbp, y_train)
            y_pred = knn.predict(X_test_lbp)
            acc = accuracy_score(y_test, y_pred)

            results.append({"P": P, "R": R, "k": k, "accuracy": acc})
            print(f"P={P}, R={R}, k={k} → accuracy={acc:.4f}")


In [6]:
import pandas as pd

In [ ]:

# -----------------------------
# Результати у таблицю
# -----------------------------
df_results = pd.DataFrame(results)
print("\nТаблиця результатів:")
print(df_results)

# -----------------------------
# Візуалізація
# -----------------------------
plt.figure(figsize=(10,6))
for (P, R), group in df_results.groupby(["P","R"]):
    plt.plot(group["k"], group["accuracy"], marker="o", label=f"P={P}, R={R}")
plt.title("Accuracy vs k for different LBP (P,R)")
plt.xlabel("k (neighbors)")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

# 2. HOG + SVM

In [5]:
from skimage.feature import hog
from sklearn.svm import LinearSVC

In [6]:
# -----------------------------
# 1. HOG
# -----------------------------
def extract_hog_features(images):
    features = []
    for img in images:
        hog_features = hog(img,
                           orientations=9,       # кількість напрямків
                           pixels_per_cell=(8,8), # розмір "клітинки"
                           cells_per_block=(2,2), # блоки для нормалізації
                           block_norm='L2-Hys',
                           transform_sqrt=True,   # покращує стійкість до освітлення
                           feature_vector=True)
        features.append(hog_features)
    return np.array(features)


In [ ]:
print("HOG train...")
X_train_hog = extract_hog_features(X_train)
print("HOG test...")
X_test_hog = extract_hog_features(X_test)

print("Shapes:", X_train_hog.shape, X_test_hog.shape)

In [ ]:
# -----------------------------
# 2. Linear SVM
# -----------------------------
svm = LinearSVC(C=0.1, max_iter=5000)
svm.fit(X_train_hog, y_train)

In [ ]:
# -----------------------------
# 3. Оцінка
# -----------------------------
y_pred = svm.predict(X_test_hog)
acc = accuracy_score(y_test, y_pred)
print("HOG + LinearSVM accuracy:", acc)

print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=label_map.keys()))

In [ ]:
# -----------------------------
# 4. Confusion matrix
# -----------------------------
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_map.keys(), yticklabels=label_map.keys())
plt.title("Confusion Matrix (HOG + Linear SVM)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
# -----------------------------
# 5. Save model
# -----------------------------
joblib.dump(svm, "hog_svm.pkl")
print("Model saved hog_svm.pkl")

## ============ Тест ===============

In [16]:
from skimage import exposure

In [ ]:
def show_hog_example(image, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2)):
    # HOG + візуалізація
    hog_features, hog_image = hog(image,
                                  orientations=orientations,
                                  pixels_per_cell=pixels_per_cell,
                                  cells_per_block=cells_per_block,
                                  block_norm='L2-Hys',
                                  transform_sqrt=True,
                                  visualize=True,
                                  feature_vector=True)

    # Масштабування для відображення
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    # Малюємо
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5), sharex=True, sharey=True)

    ax1.imshow(image, cmap=plt.cm.gray)
    ax1.set_title("Оригінал")

    ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
    ax2.set_title(f"HOG (orient={orientations}, cell={pixels_per_cell}, block={cells_per_block})")

    plt.show()

# Приклад: подивимось на одне фото з train
sample_img = X_train[0]
show_hog_example(sample_img, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2))
show_hog_example(sample_img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2,2))
show_hog_example(sample_img, orientations=12, pixels_per_cell=(8,8), cells_per_block=(3,3))


In [ ]:
configs = [
    {"orientations": 9, "pixels_per_cell": (8,8), "cells_per_block": (2,2)},
    {"orientations": 9, "pixels_per_cell": (4,4), "cells_per_block": (2,2)},
    {"orientations": 12, "pixels_per_cell": (8,8), "cells_per_block": (3,3)},
    {"orientations": 6, "pixels_per_cell": (16,16), "cells_per_block": (2,2)},
]

results = []

for cfg in configs:
    print(f"Параметри: {cfg}")
    # HOG ознаки
    X_train_hog = [hog(img,
                       orientations=cfg["orientations"],
                       pixels_per_cell=cfg["pixels_per_cell"],
                       cells_per_block=cfg["cells_per_block"],
                       block_norm='L2-Hys',
                       transform_sqrt=True,
                       feature_vector=True) for img in X_train]
    X_test_hog = [hog(img,
                      orientations=cfg["orientations"],
                      pixels_per_cell=cfg["pixels_per_cell"],
                      cells_per_block=cfg["cells_per_block"],
                      block_norm='L2-Hys',
                      transform_sqrt=True,
                      feature_vector=True) for img in X_test]

    X_train_hog = np.array(X_train_hog)
    X_test_hog = np.array(X_test_hog)

    # SVM
    svm = LinearSVC(C=0.1, max_iter=5000)
    svm.fit(X_train_hog, y_train)
    y_pred = svm.predict(X_test_hog)
    acc = accuracy_score(y_test, y_pred)

    results.append({"orientations": cfg["orientations"],
                    "pixels_per_cell": cfg["pixels_per_cell"],
                    "cells_per_block": cfg["cells_per_block"],
                    "accuracy": acc})
    print(f"Accuracy = {acc:.4f}")

df = pd.DataFrame(results)
print("\nРезультати експерименту:")
print(df)


# CNN

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, Conv2D, SeparableConv2D, BatchNormalization,
                                     Activation, GlobalAveragePooling2D, Add)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_dir = "/kaggle/input/fer2013/train"
val_dir = "/kaggle/input/fer2013/test"

# --------------------------
# 1. Data Generators
# --------------------------
img_size = (48, 48)
batch_size = 64

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True
# )

# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=15,
#     width_shift_range=0.15,
#     height_shift_range=0.15,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True
# )

from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
])


val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_classes = train_gen.num_classes

In [15]:
# --------------------------
# 2. mini-Xception Architecture
# --------------------------
def mini_xception(input_shape=(48,48,1), num_classes=7):
    inputs = Input(shape=input_shape)

    # Entry flow
    x = Conv2D(8, (3,3), strides=(1,1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(8, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    residual = Conv2D(16, (1,1), strides=(2,2), padding='same')(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(16, (3,3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(16, (3,3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    residual = Conv2D(32, (1,1), strides=(2,2), padding='same')(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(32, (3,3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(32, (3,3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    residual = Conv2D(64, (1,1), strides=(2,2), padding='same')(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(64, (3,3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = SeparableConv2D(64, (3,3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same')(x)
    x = Add()([x, residual])

    # Final layers
    x = Conv2D(num_classes, (3,3), padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    outputs = Activation('softmax')(x)

    model = Model(inputs, outputs)
    return model

In [16]:
# --------------------------
# 3. Compile & Train
# --------------------------
model = mini_xception(input_shape=(48,48,1), num_classes=num_classes)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

#model.summary()

In [17]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights = dict(enumerate(class_weights))


In [18]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint("best_mini_xception.keras", monitor="val_loss", save_best_only=True)
]


In [ ]:
# --------------------------
# 4. Training
# --------------------------
epochs = 50

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=callbacks
)

In [ ]:
# оцінка на валідаційному наборі
val_loss, val_acc = model.evaluate(val_gen, verbose=1)
print(f"Validation accuracy: {val_acc*100:.2f}%")

# якщо є окремий test набір
test_loss, test_acc = model.evaluate(val_gen, verbose=1)
print(f"Test accuracy: {test_acc*100:.2f}%")

In [11]:
# --------------------------
# 5. Save model
# --------------------------
model.save("mini_xception_fer2013_2.keras")

In [ ]:
# --------------------------
# 6. Plot results
# --------------------------
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train_acc")
plt.plot(history.history["val_accuracy"], label="val_acc")
plt.legend()
plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend()
plt.title("Loss")
plt.show()
